In [99]:
import pandas as pd
import numpy as np

In [100]:
# 보험 청구 금액 예측하는 인공지능 개발

In [101]:
# 1. 데이터 로드 및 전처리
df=pd.read_csv('data/healthcare.csv',index_col=0)

In [102]:
df.head(2)

,Age,Gender,BMI,Region,Smoker,NumVisits,InsuranceClaim
0,51.0,Female,NaN,South,No,19.0,70.081987
1,92.0,Female,38.074006,West,Yes,13.0,92.747518


In [103]:
# 타겟 변수와 피처 분리
y=df['InsuranceClaim']
X=df.drop(columns=['InsuranceClaim'])

In [104]:
y

0      70.081987
1      92.747518
2      46.794138
3      44.789132
4      71.790344
         ...    
495    79.352997
496    64.364345
497    74.631320
498    75.066459
499    45.184790
Name: InsuranceClaim, Length: 500, dtype: float64

In [105]:
X

,Age,Gender,BMI,Region,Smoker,NumVisits
0,51.0,Female,NaN,South,No,19.0
1,92.0,Female,38.074006,West,Yes,13.0
2,14.0,Male,NaN,North,No,8.0
3,NaN,Female,27.020924,West,No,7.0
4,60.0,Male,37.961368,North,No,16.0
...,...,...,...,...,...,...
495,95.0,Male,29.831966,East,No,20.0
496,47.0,Female,31.257355,West,No,13.0
497,88.0,Female,26.981973,South,Yes,11.0
498,0.0,Male,30.393551,North,Yes,20.0


In [106]:
X.isna().sum()

Age          30
Gender        0
BMI          50
Region        0
Smoker        0
NumVisits    20
dtype: int64

In [107]:
# 2. 카테고리컬 데이터와 수치형 데이터 처리

In [108]:
# 컬러 이름을 분리해서 저장

In [109]:
# 수치형 컬럼 이름
numerrical=['Age','BMI','NumVisits']

In [110]:
# 카테고리컬 데이터 컬럼 이름
categorical=['Gender','Region','Smoker']

In [111]:
# 수치형 데이터의 NaN을, 각 컬럼의 평균값으로 채운다.
# 카테고리컬 데이터는 레이블인코딩, 원핫인코딩한다.

In [112]:
# sklearn의 pipline 라이브어잉 사용할 것이니까
# NaN을 채우는 것은, fillna 함수 대신에, SimpleImputer를 사용한다.

In [113]:
# 수치형 컬럼의 NaN을 각 컬럼의 평균으로 채우는 것
from sklearn.impute import SimpleImputer

imputer=SimpleImputer()

In [114]:
from sklearn.preprocessing import OrdinalEncoder,OneHotEncoder

In [115]:
labe_encoder=OrdinalEncoder()
onehot_encoder=OneHotEncoder()

In [116]:
from sklearn.compose import ColumnTransformer

In [117]:
df

,Age,Gender,BMI,Region,Smoker,NumVisits,InsuranceClaim
0,51.0,Female,NaN,South,No,19.0,70.081987
1,92.0,Female,38.074006,West,Yes,13.0,92.747518
2,14.0,Male,NaN,North,No,8.0,46.794138
3,NaN,Female,27.020924,West,No,7.0,44.789132
4,60.0,Male,37.961368,North,No,16.0,71.790344
...,...,...,...,...,...,...,...
495,95.0,Male,29.831966,East,No,20.0,79.352997
496,47.0,Female,31.257355,West,No,13.0,64.364345
497,88.0,Female,26.981973,South,Yes,11.0,74.631320
498,0.0,Male,30.393551,North,Yes,20.0,75.066459


In [118]:
df['Gender'].nunique()

2

In [119]:
df['Region'].nunique()

4

In [120]:
df['Smoker'].nunique()

2

In [121]:

label_columns=['Gender','Smoker']

In [122]:
onehot_columns=['Region']

In [123]:
numeric_colums=['Age','BMI','NumVisits']

In [124]:
# 각 전처리할 라이브러리 준비하고
# 해당 라이브러리에 적용할 컬럼리스트를 준비한다.

In [125]:
preprocessor=ColumnTransformer([
    ('num',imputer,numeric_colums),
    ('lable',labe_encoder,label_columns),
    ('onehot',onehot_encoder,onehot_columns)
])

In [126]:
# 3. 파이프라인(pipeline) 생성

In [127]:
from sklearn.ensemble import RandomForestRegressor

In [128]:
regressor=RandomForestRegressor(random_state=42)

In [129]:
from sklearn.pipeline import Pipeline

In [135]:
Pipeline=Pipeline(steps=[('preprocessing',preprocessor),('modeling',regressor)])

In [131]:
# 학습/테스트 데이터 분리 후 학습

In [132]:
from sklearn.model_selection import train_test_split

In [133]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [136]:
Pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Age', 'BMI', 'NumVisits']),
                                                 ('lable', OrdinalEncoder(),
                                                  ['Gender', 'Smoker']),
                                                 ('onehot', OneHotEncoder(),
                                                  ['Region'])])),
                ('modeling', RandomForestRegressor(random_state=42))])

In [137]:
y_pred=Pipeline.predict(X_test)

In [138]:
from sklearn.metrics import mean_squared_error,r2_score

In [139]:
mean_squared_error(y_test,y_pred)

46.85468358473401

In [140]:
r2_score(y_test,y_pred)

0.7527759919881363

In [ ]:
# 파이프라인을 이용하면 좋은점!!

# 이 파이프라인만 파일로 저장하면, 서비스 배포할때 이 파일만 있으면,
# 예측하는 데이터만 그냥 넣어주면, 파이프라인이 내부적으로 데이터 전처리를 해서 결과를 안려준다.

In [141]:
import joblib

In [142]:
joblib.dump(Pipeline,'pipeline.pkl')

['pipeline.pkl']